In [ ]:
from google.colab import drive
drive.mount('/content/drive' , force_remount = True)

Mounted at /content/drive


#### Import dependencies

In [ ]:
!pip install --upgrade optuna optkeras
!pip install mcfly

import os
import statistics
import csv
import itertools
import tensorflow as tf
import keras
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sn
from tqdm import tqdm_notebook as tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import *
from keras.utils import to_categorical
from keras.optimizers import Adam
from tensorflow.keras.utils import plot_model
import mcfly
from keras.utils import plot_model
import optuna
import optkeras
from numpy import std, mean
optuna.logging.set_verbosity(optuna.logging.WARN)

In [ ]:
cd "/content/drive/MyDrive/PAMAP2_Dataset"

/content/drive/MyDrive/PAMAP2_Dataset


### MHEALTH

In [ ]:
cd "/content/drive/MyDrive/MHEALTHDATASET/"

/content/drive/MyDrive/MHEALTHDATASET


In [ ]:
df_mhealth = pd.read_pickle('mhealth_cnn.pkl')

#### GKF split on last 10% of records per subject

In [ ]:
grouped_data = df_mhealth.groupby('subject')
train_data = pd.DataFrame()
val_data = pd.DataFrame()
test_data = pd.DataFrame()
for _, group in grouped_data:
    sorted_group = group.sort_index()

    total_samples = len(sorted_group)
    train_size = int(0.7 * total_samples)
    val_size = int(0.15 * total_samples)

    train_group = sorted_group.iloc[val_size:val_size + train_size]
    val_group = sorted_group.iloc[train_size + val_size:]
    test_group = sorted_group.iloc[:val_size]

    train_data = pd.concat([train_data, train_group])
    val_data = pd.concat([val_data, val_group])
    test_data = pd.concat([test_data, test_group])

##### sliding windows

In [ ]:
window_size = 200
stride = 50

train_frames = []
for i in tqdm(range(0, len(train_data)-window_size, stride)):
    window = train_data.iloc[i:i+window_size]
    if window['activity'].nunique() == 1:
      train_frames.append(window)

test_frames = []
for i in tqdm(range(0, len(test_data)-window_size, stride)):
    window = test_data.iloc[i:i+window_size]
    if window['activity'].nunique() == 1:
      test_frames.append(window)

val_frames = []
for i in tqdm(range(0, len(val_data)-window_size, stride)):
    window = val_data.iloc[i:i+window_size]
    if window['activity'].nunique() == 1:
      val_frames.append(window)

<ipython-input-50-b0637a775a85>:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(range(0, len(train_data)-window_size, stride)):


  0%|          | 0/5231 [00:00<?, ?it/s]

<ipython-input-50-b0637a775a85>:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(range(0, len(test_data)-window_size, stride)):


  0%|          | 0/1118 [00:00<?, ?it/s]

<ipython-input-50-b0637a775a85>:17: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(range(0, len(val_data)-window_size, stride)):


  0%|          | 0/1118 [00:00<?, ?it/s]

In [ ]:
X_train_list = []
y_train_list = []
relevant_columns = [col for col in train_frames[0].columns if col not in ['subject', 'activity']]

#for each frame replace label with activity
for frame in tqdm(train_frames):
    X_train_list.append(frame[relevant_columns].values)
    y_train_list.append(frame.iloc[0]['activity'])

X_test_list = []
y_test_list = []
for frame in tqdm(test_frames):
    X_test_list.append(frame[relevant_columns].values)
    y_test_list.append(frame.iloc[0]['activity'])

X_val_list = []
y_val_list = []
for frame in tqdm(val_frames):
    X_val_list.append(frame[relevant_columns].values)
    y_val_list.append(frame.iloc[0]['activity'])

<ipython-input-51-6f01deab48dd>:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for frame in tqdm(train_frames):


  0%|          | 0/4645 [00:00<?, ?it/s]

<ipython-input-51-6f01deab48dd>:12: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for frame in tqdm(test_frames):


  0%|          | 0/957 [00:00<?, ?it/s]

<ipython-input-51-6f01deab48dd>:18: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for frame in tqdm(val_frames):


  0%|          | 0/923 [00:00<?, ?it/s]

In [ ]:
X_train = np.array(X_train_list)
y_train = np.array(to_categorical(y_train_list, num_classes=13))

X_test = np.array(X_test_list)
y_test = np.array(to_categorical(y_test_list, num_classes=13))

X_val = np.array(X_val_list)
y_val = np.array(to_categorical(y_val_list, num_classes=13))

In [ ]:
#fit scaler on train
scaler = StandardScaler()
train_reshaped = X_train.reshape(-1, X_train.shape[-1])
X_train_scaled = scaler.fit_transform(train_reshaped)
X_train = X_train_scaled.reshape(X_train.shape)

In [ ]:
val_reshaped = X_val.reshape(-1, X_val.shape[-1])
X_val_scaled = scaler.transform(val_reshaped)
X_val = X_val_scaled.reshape(X_val.shape)

test_reshaped = X_test.reshape(-1, X_test.shape[-1])
X_test_scaled = scaler.transform(test_reshaped)
X_test = X_test_scaled.reshape(X_test.shape)

In [ ]:
print("X_train: ",X_train.shape)
print("X_val: ",X_val.shape)
print("X_test: ",X_test.shape)

print("Y_train: ",y_train.shape)
print("Y_val: ",y_val.shape)
print("Y_test: ",y_test.shape)

X_train:  (4630, 200, 23)
X_val:  (988, 200, 23)
X_test:  (923, 200, 23)
Y_train:  (4630, 13)
Y_val:  (988, 13)
Y_test:  (923, 13)


In [ ]:
n_steps, n_length,n_features = 4,50,23
trainX = X_train.reshape((X_train.shape[0], n_steps, n_length, n_features))
valX = X_val.reshape((X_val.shape[0], n_steps, n_length, n_features))
testX = X_test.reshape((X_test.shape[0], n_steps, n_length, n_features))

In [ ]:
from scipy.stats import entropy
import numpy as n

# Compute Jensen-Shannon divergence for each feature
js_divergences = np.zeros(trainX.shape[1])
for i in range(trainX.shape[1]):
    # Compute histograms with the same bins for both X_train and X_test
    bins = max(len(np.unique(trainX[:, i])), len(np.unique(testX[:, i])))
    p, _ = np.histogram(trainX[:, i], bins=bins, density=True)
    q, _ = np.histogram(testX[:, i], bins=bins, density=True)
    m = 0.5 * (p + q)
    js_divergences[i] = 0.5 * (entropy(p, m) + entropy(q, m))

avg_js_divergence = np.mean(js_divergences)
print("Average Jensen-Shannon divergence:", avg_js_divergence)

Average Jensen-Shannon divergence: 0.3698580423757989


### PAMAP2 DATASET Preproc

#### GKF split.
Similarly to LSTM, 170k for train, rest split for val/test

In [ ]:
data = pd.read_pickle('pamap2.df')

In [ ]:
from sklearn.model_selection import train_test_split
grouped_data = data.groupby('subject_id')
train_data = pd.DataFrame()
val_data = pd.DataFrame()
test_data = pd.DataFrame()

for _, group in grouped_data:
    sorted_group = group.sort_values(by='time_stamp', ascending=True)

    #train has first 170k rows
    train_group = sorted_group.iloc[15000:145000]
    val_group = sorted_group.iloc[:15000]
    test_group = sorted_group.iloc[145000:]
    #train_group = train_group[::10]
    #val_group = val_group[::10]
    #test_group = test_group[::10]

    train_data = pd.concat([train_data, train_group])
    test_data = pd.concat([test_data, test_group])
    val_data = pd.concat([val_data, val_group])

In [ ]:
from sklearn.model_selection import train_test_split

grouped_data = data.groupby('subject_id')
train_data = pd.DataFrame()
val_data = pd.DataFrame()
test_data = pd.DataFrame()

for _, group in grouped_data:
    sorted_group = group.sort_values(by='time_stamp', ascending=True)

    total_samples = len(sorted_group)
    train_size = int(0.6 * total_samples)
    val_size = int(0.2 * total_samples)

    train_group = sorted_group.iloc[:train_size]
    val_group = sorted_group.iloc[train_size:train_size + val_size]
    test_group = sorted_group.iloc[train_size + val_size:]

    train_data = pd.concat([train_data, train_group])
    val_data = pd.concat([val_data, val_group])
    test_data = pd.concat([test_data, test_group])

In [ ]:
print(train_data["activity_id"].value_counts())
print(val_data["activity_id"].value_counts())

#### Create sliding windows from samples

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(train_data)
train_data_scaled = scaler.transform(train_data)
val_data_scaled = scaler.transform(val_data)
test_data_scaled = scaler.transform(test_data)

train_data = pd.DataFrame(train_data_scaled, columns=train_data.columns)
val_data = pd.DataFrame(val_data_scaled, columns=val_data.columns)
test_data = pd.DataFrame(test_data_scaled, columns=test_data.columns)

In [ ]:
len(train_frames)

In [ ]:
window_size = 200
stride = 50

train_frames = []
for i in tqdm(range(0, len(train_data)-window_size, stride)):
    window = train_data.iloc[i:i+window_size]
    if window['activity_id'].nunique() == 1:
      train_frames.append(window)

test_frames = []
for i in tqdm(range(0, len(test_data)-window_size, stride)):
    window = test_data.iloc[i:i+window_size]
    if window['activity_id'].nunique() == 1:
      test_frames.append(window)

val_frames = []
for i in tqdm(range(0, len(val_data)-window_size, stride)):
    window = val_data.iloc[i:i+window_size]
    if window['activity_id'].nunique() == 1:
      val_frames.append(window)

<ipython-input-19-80ac3ef01d65>:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(range(0, len(train_data)-window_size, stride)):


  0%|          | 0/20796 [00:00<?, ?it/s]

<ipython-input-19-80ac3ef01d65>:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(range(0, len(test_data)-window_size, stride)):


  0%|          | 0/15526 [00:00<?, ?it/s]

<ipython-input-19-80ac3ef01d65>:17: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(range(0, len(val_data)-window_size, stride)):


  0%|          | 0/2396 [00:00<?, ?it/s]

In [ ]:
len(val_frames)

0

In [ ]:
X_train_list = []
y_train_list = []
relevant_columns = [col for col in train_frames[0].columns if col not in ['subject_id', 'activity_id', 'time_ste[]']]

#for each frame replace label with activity
for frame in tqdm(train_frames):
    X_train_list.append(frame[relevant_columns].values)
    y_train_list.append(frame.iloc[0]['activity_id'])

X_test_list = []
y_test_list = []
#relevant_columns = [col for col in frames.columns if col not in ['subject_id', 'activity_id']]
relevant_columns = [col for col in test_frames[0].columns if col not in ['subject_id', 'activity_id', 'time_ste[]']]

#for each frame replace label with activity
for frame in tqdm(test_frames):
    X_test_list.append(frame[relevant_columns].values)
    y_test_list.append(frame.iloc[0]['activity_id'])

X_val_list = []
y_val_list = []
#relevant_columns = [col for col in frames.columns if col not in ['subject_id', 'activity_id']]
relevant_columns = [col for col in val_frames[0].columns if col not in ['subject_id', 'activity_id', 'time_ste[]']]

#for each frame replace label with activity
for frame in tqdm(val_frames):
    X_val_list.append(frame[relevant_columns].values)
    y_val_list.append(frame.iloc[0]['activity_id'])

<ipython-input-20-360b403d9f03>:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for frame in tqdm(train_frames):


  0%|          | 0/20527 [00:00<?, ?it/s]

<ipython-input-20-360b403d9f03>:16: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for frame in tqdm(test_frames):


  0%|          | 0/15356 [00:00<?, ?it/s]

<ipython-input-20-360b403d9f03>:26: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for frame in tqdm(val_frames):


  0%|          | 0/2396 [00:00<?, ?it/s]

In [ ]:
X_train = np.array(X_train_list)
y_train = np.array(to_categorical(y_train_list, num_classes=25))

X_test = np.array(X_test_list)
y_test = np.array(to_categorical(y_test_list, num_classes=25))

X_val = np.array(X_val_list)
y_val = np.array(to_categorical(y_val_list, num_classes=25))

In [ ]:
print("X_train: ",X_train.shape)
print("X_val: ",X_val.shape)
print("X_test: ",X_test.shape)

print("Y_train: ",y_train.shape)
print("Y_val: ",y_val.shape)
print("Y_test: ",y_test.shape)

In [ ]:
n_steps, n_length,n_features = 4,50,21
trainX = X_train.reshape((X_train.shape[0], n_steps, n_length, n_features))
valX = X_val.reshape((X_val.shape[0], n_steps, n_length, n_features))
testX = X_test.reshape((X_test.shape[0], n_steps, n_length, n_features))

In [ ]:
from scipy.stats import entropy
import numpy as np

# Compute Jensen-Shannon divergence for each feature
js_divergences = np.zeros(trainX.shape[1])
for i in range(trainX.shape[1]):
    # Compute histograms with the same bins for both X_train and X_test
    bins = max(len(np.unique(trainX[:, i])), len(np.unique(testX[:, i])))
    p, _ = np.histogram(trainX[:, i], bins=bins, density=True)
    q, _ = np.histogram(testX[:, i], bins=bins, density=True)
    m = 0.5 * (p + q)
    js_divergences[i] = 0.5 * (entropy(p, m) + entropy(q, m))

# Average JS divergence across all features
avg_js_divergence = np.mean(js_divergences)
print("Average Jensen-Shannon divergence:", avg_js_divergence)

Average Jensen-Shannon divergence: 0.698837643217995


### Actual model

In [ ]:
model = Sequential()
model.add(TimeDistributed(Conv1D(filters=96, kernel_size=3, activation='relu'), input_shape=(None,n_length,n_features)))
model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
model.add(TimeDistributed(Conv1D(filters=128, kernel_size=3, activation='relu')))
model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
model.add(TimeDistributed(Dropout(0.06)))
model.add(TimeDistributed(Flatten()))
model.add(LSTM(64 , return_sequences= True ))
model.add(LSTM(64))
model.add(Dropout(0.1))
model.add(Dense(y_train.shape[1], activation = 'softmax'))


In [ ]:
plot_model(model, to_file='model1_plot.png', show_shapes=True,rankdir = 'TB',expand_nested = False,  show_layer_names=False, show_dtype =False)

In [ ]:
model.compile(Adam(),
              loss=tf.keras.losses.categorical_crossentropy,
              metrics=['acc'])

In [ ]:
history = model.fit(trainX,y_train, epochs=8, validation_data=(valX, y_val), verbose=1 , batch_size  = 128 )

Epoch 1/8
37/37 [==============================] - 8s 120ms/step - loss: 1.0359 - acc: 0.7684 - val_loss: 2.9466 - val_acc: 0.3824
Epoch 2/8
37/37 [==============================] - 4s 98ms/step - loss: 0.1223 - acc: 0.9737 - val_loss: 2.7230 - val_acc: 0.3900
Epoch 3/8
37/37 [==============================] - 4s 119ms/step - loss: 0.0527 - acc: 0.9912 - val_loss: 3.0984 - val_acc: 0.3944
Epoch 4/8
37/37 [==============================] - 4s 97ms/step - loss: 0.0230 - acc: 0.9981 - val_loss: 2.9461 - val_acc: 0.4410
Epoch 5/8
37/37 [==============================] - 4s 97ms/step - loss: 0.0195 - acc: 0.9970 - val_loss: 3.2835 - val_acc: 0.4095
Epoch 6/8
37/37 [==============================] - 4s 119ms/step - loss: 0.0189 - acc: 0.9961 - val_loss: 3.5634 - val_acc: 0.3824
Epoch 7/8
37/37 [==============================] - 4s 97ms/step - loss: 0.0083 - acc: 0.9996 - val_loss: 3.4132 - val_acc: 0.4247
Epoch 8/8
37/37 [==============================] - 4s 97ms/step - loss: 0.0062 - acc: 0

In [ ]:
model.save("gupta_gkf.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


### predict on test

In [ ]:
#load model
loaded_model = tf.keras.models.load_model("gupta_gkf.h5")

In [ ]:
lstm_y_test_pred = loaded_model.predict(testX)
y_test_argmax = y_test.argmax(axis=1)
y_test_pred_argmax = lstm_y_test_pred.argmax(axis=1)

30/30 [==============================] - 1s 9ms/step


In [ ]:
og_test = np.argmax(y_test, axis=1)

In [ ]:
print("Smartphone Results with CNN-GRU for Test Set\n\n" + classification_report(y_test_pred_argmax,og_test , digits= 4))